# Prompting flan-T5-base

In [ ]:
%%capture
!pip install -U datasets
!pip install transformers datasets evaluate rouge_score --quiet
!pip uninstall keras -y
!pip install keras==2.11

In [ ]:
import evaluate
from datasets import load_dataset
from tqdm import tqdm
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

2025-06-19 08:06:03.771810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750320363.799007     233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750320363.808378     233 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
# Full dataset (split included)
dataset = load_dataset("EdinburghNLP/xsum")
test_sample = dataset["test"][0]

In [ ]:
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
# def generate_with_strategy(doc, strategy="greedy"):
#     input_ids = tokenizer("summarize: " + doc, return_tensors="pt", truncation=True).input_ids.to(model.device)

#     if strategy == "greedy":
#         output = model.generate(input_ids, max_length=64)
#     elif strategy == "beam":
#         output = model.generate(input_ids, max_length=64, num_beams=5, early_stopping=True)
#     elif strategy == "topk":
#         output = model.generate(input_ids, max_length=64, do_sample=True, top_k=50)
#     elif strategy == "topp":
#         output = model.generate(input_ids, max_length=64, do_sample=True, top_p=0.9)
#     else:
#         raise ValueError("Unknown strategy")

#     return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# strategies = ["greedy", "beam", "topk", "topp"]
# subset = dataset["test"].select(range(100))
# references = [example["summary"] for example in subset]


# for i in range(3):  # Test on 3 samples
#     doc = subset[i]["document"]
#     ref = subset[i]["summary"]

#     print(f"\n🔹 Example {i+1}")
#     print("="*80)
#     print("📄 Document:\n", doc[:800], "..." if len(doc) > 800 else "")
#     print("\n✅ Reference Summary:\n", ref)

#     for strat in strategies:
#         pred = generate_with_strategy(doc, strategy=strat)
#         print(f"\n🧠 {strat.upper()} Output:\n{pred}")
#     print("="*80)

In [ ]:
# def generate_summary(doc, prompt_template="Please write a short summary of the following article:\n\n{}",
#                      max_input=512, max_output=64):
#     prompt = prompt_template.format(doc)
#     inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_input, padding=True).to(model.device)
#     outputs = model.generate(**inputs, max_length=max_output)
#     return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# rouge = evaluate.load("rouge")

# # Evaluate on a 100-sample test slice
# subset = dataset["test"].select(range(3))

# predictions = [generate_summary(example["document"]) for example in subset]
# references = [example["summary"] for example in subset]

# rouge_results = rouge.compute(predictions=predictions, references=references)
# print("ROUGE Results:", rouge_results)

In [ ]:
# # Show a few test examples with model predictions
# num_examples = 3
# for i in range(num_examples):
#     doc = subset[i]["document"]
#     ref = subset[i]["summary"]
#     pred = generate_summary(doc)

#     print(f"\n🔹 Example {i+1}")
#     print("=" * 80)
#     print("📄 Document:\n", doc[:800], "..." if len(doc) > 800 else "")  # truncate long doc
#     print("\n✅ Reference Summary:\n", ref)
#     print("🤖 Model Summary:\n", pred)
#     print("=" * 80)

### Few shot

In [ ]:
# Few-shot examples from training set
train_examples = dataset["train"].select(range(2))

def build_few_shot_prompt(target_doc, few_shots):
    prompt = ""
    for ex in few_shots:
        prompt += f"Document: {ex['document'].strip()}\nSummary: {ex['summary'].strip()}\n\n"
    prompt += f"Document: {target_doc.strip()}\nSummary:"
    return prompt

In [ ]:
def generate_few_shot(doc, few_shots):
    prompt = build_few_shot_prompt(doc, few_shots)
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_length=64)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
# try:
#   from google.colab import drive
#   # Mount Google Drive
#   drive.mount('/content/drive')
#   print("Google Drive connected successfully!")
# except ImportError:
#   print("Not in Google Colab. Skipping Google Drive connection.")

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

'cuda:0'

In [ ]:
# Move the model to the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
# Generate few-shot predictions in batches
batch_size = 20
few_shot_preds = []

# Select a larger subset for demonstration
subset = dataset["test"]

for i in tqdm(range(0, len(subset), batch_size)):
    batch_subset = []
    for j in range(i, min(i + batch_size, len(subset))):
        batch_subset.append(subset[j])
    batch_docs = [example["document"] for example in batch_subset]

    batch_prompts = [build_few_shot_prompt(doc, train_examples) for doc in batch_docs]

    # Tokenize and move inputs to the correct device
    inputs = tokenizer(batch_prompts, return_tensors="pt", truncation=True, max_length=2024, padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}  # Fix: move input tensors to the device

    # Generate predictions
    outputs = model.generate(**inputs, max_length=64)  # Do NOT call .to(device) here

    # Decode predictions
    batch_preds = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    few_shot_preds.extend(batch_preds)

100%|██████████| 567/567 [1:33:43<00:00,  9.92s/it]


In [ ]:
len(few_shot_preds)

11334

In [ ]:
# # Generate few-shot predictions in batches
# batch_size = 5
# few_shot_preds = []

# # Select a larger subset for demonstration
# subset = dataset["test"].select(range(10))

# for i in tqdm(range(0, len(subset), batch_size)):
#     batch_subset = []
#     for j in range(i, i+batch_size):
#       batch_subset.append(subset[j])
#     batch_docs = [example["document"] for example in batch_subset]

#     batch_prompts = [build_few_shot_prompt(doc, train_examples) for doc in batch_docs]

#     inputs = tokenizer(batch_prompts, return_tensors="pt", truncation=True, max_length=2024, padding=True).to(device)
#     outputs = model.generate(**inputs, max_length=64).to(device)

#     batch_preds = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
#     few_shot_preds.extend(batch_preds)

In [ ]:
# input = []
# subset = dataset["test"].select(range(10))

# for i in range(len(subset)):
#   doc = subset[i]["document"]
#   few = train_examples
#   input.append(build_few_shot_prompt(doc,few))

# inputs = tokenizer(input, return_tensors="pt", truncation=True, max_length=2024, padding=True).to(model.device)
# outputs = model.generate(**inputs, max_length=64)
# ops = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
# ops

In [ ]:
import pickle
import os # Import the os module to work with file paths

# Define the path in your Google Drive where you want to save the file.
# '/content/drive/My Drive/' is the root of your Google Drive.
# 'my_project_data' is a suggested folder name within your Drive.
# 'ols_data.pkl' is the filename for the pickle file.
drive_path = '/kaggle/working/'
file_name = 'few_shot_testset.pkl'
full_path = os.path.join(drive_path, file_name)

# Ensure the directory exists (optional but good practice)
# This will create the 'my_project_data' folder if it doesn't exist
os.makedirs(drive_path, exist_ok=True)

# Open the file in binary write mode ('wb') and use pickle.dump()
try:
    with open(full_path, 'wb') as f:
        pickle.dump(few_shot_preds, f)
    print(f"Successfully saved the list as pickle to: {full_path}")
except Exception as e:
    print(f"An error occurred while saving the pickle file: {e}")

Successfully saved the list as pickle to: /kaggle/working/few_shot_testset.pkl


In [ ]:
# Generate few-shot predictions (dynamic prompt per example)
few_shot_preds = []



for idx, example in tqdm(enumerate(subset)):
    summary = generate_few_shot(example["document"], train_examples)
    few_shot_preds.append(summary)
    if idx >= 2:
      break

few_shot_rouge = rouge.compute(predictions=few_shot_preds, references=references)
print("Few-shot ROUGE:", few_shot_rouge)

NameError: name 'tqdm' is not defined

In [ ]:
for i in range(3):
    print(f"\n🔹 Example {i+1}")
    print("="*80)
    print("📄 Document:\n", subset[i]["document"][:800], "..." if len(subset[i]["document"]) > 800 else "")
    print("\n✅ Reference Summary:\n", subset[i]["summary"])
    print("🤖 Few-shot Summary:\n", few_shot_preds[i])
    print("="*80)


🔹 Example 1
📄 Document:
 Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.
Workers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.
The Welsh Government said more people than ever were getting help to address housing problems.
Changes to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.
Prison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.
However, the same could not be said for men, the charity said, because issues which often affect them, such as post tra ...

✅ Reference Summary:
 There is a "chronic" need for more housing for prison leavers in Wales, according to a charity.
🤖 Few-shot Summary:
 The Dumfries and Galloway are

### BertScore

In [ ]:
!pip install bert-score

In [ ]:
from bert_score import score

In [ ]:
P, R, F1 = score(few_shot_preds, references, lang="en", verbose=True)
print("BERTScore:")
print(f"  Precision: {P.mean().item():.4f}")
print(f"  Recall: {R.mean().item():.4f}")
print(f"  F1 Score: {F1.mean().item():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 6.84 seconds, 0.44 sentences/sec
BERTScore:
  Precision: 0.8544
  Recall: 0.8602
  F1 Score: 0.8573


In [ ]:
P_few_shot, R_few_shot, F1_few_shot = score(few_shot_preds, references, lang="en", verbose=True)
print("Few-shot BERTScore:")
print(f"  Precision: {P_few_shot.mean().item():.4f}")
print(f"  Recall: {R_few_shot.mean().item():.4f}")
print(f"  F1 Score: {F1_few_shot.mean().item():.4f}")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 5.52 seconds, 0.54 sentences/sec
Few-shot BERTScore:
  Precision: 0.8544
  Recall: 0.8602
  F1 Score: 0.8573


In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
sentence  = 'Hello World!'
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')
model     = BertModel.from_pretrained('bert-large-uncased')

inputs    = tokenizer(sentence, return_tensors="pt").to(device)
model     = model.to(device)
outputs   = model(**inputs)